<a href="https://colab.research.google.com/github/Pratikshaprabhakarbande/cifar-env-check/blob/main/Copy_of_Adversarial_Attacks_FGSM_PGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚀 STEP 1: COMPLETE SETUP (NO ERRORS)
# 1. Install packages
!pip install torch torchvision adversarial-robustness-toolbox tqdm --quiet

# 2. Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import os

# 3. Create folders
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('data', exist_ok=True)

# 4. Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# 5. Check GPU
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("✅ Using CPU")

✅ Using device: cpu
✅ Using CPU


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN().to(device)
print("✅ Model created successfully!")
print(f"✅ Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

✅ Model created successfully!
✅ Number of parameters: 545,098


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_indices = list(range(1000))
test_indices = list(range(200))

train_subset = Subset(trainset, train_indices)
test_subset = Subset(testset, test_indices)

trainloader = DataLoader(train_subset, batch_size=32, shuffle=True)
testloader = DataLoader(test_subset, batch_size=32, shuffle=False)

print(f"✅ Training samples: {len(train_subset)}")
print(f"✅ Test samples: {len(test_subset)}")

✅ Training samples: 1000
✅ Test samples: 200


In [ ]:
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("🚀 Starting BASIC training...")
losses = []

for epoch in range(2):
    model.train()
    total_loss = 0

    progress_bar = tqdm(trainloader, desc=f'Epoch {epoch+1}/2')

    for batch_idx, (data, target) in enumerate(progress_bar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(trainloader)
    losses.append(avg_loss)
    print(f'✅ Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}')

print("🎉 Basic training completed successfully!")

🚀 Starting BASIC training...


Epoch 1/2: 100%|██████████| 32/32 [00:01<00:00, 16.28it/s, Loss=2.2944]


✅ Epoch 1 completed. Average Loss: 2.1916


Epoch 2/2: 100%|██████████| 32/32 [00:02<00:00, 10.84it/s, Loss=1.5734]

✅ Epoch 2 completed. Average Loss: 1.8631
🎉 Basic training completed successfully!


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = outputs.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

accuracy = 100 * correct / total
print(f"✅ Basic Model Accuracy: {accuracy:.2f}%")

✅ Basic Model Accuracy: 34.00%


In [ ]:
!pip install adversarial-robustness-toolbox --quiet
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
print("✅ ART installed successfully!")

✅ ART installed successfully!


In [ ]:
art_classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    input_shape=(3, 32, 32),
    nb_classes=10,
    clip_values=(-1, 1),
    device_type='cpu'
)

data, target = next(iter(testloader))
data, target = data.to(device), target.to(device)

attack = FastGradientMethod(estimator=art_classifier, eps=0.1)
data_adv_np = attack.generate(x=data.cpu().numpy())
data_adv = torch.from_numpy(data_adv_np).to(device)

with torch.no_grad():
    outputs_clean = model(data)
    outputs_adv = model(data_adv)

    _, pred_clean = outputs_clean.max(1)
    _, pred_adv = outputs_adv.max(1)

    clean_acc = pred_clean.eq(target).sum().item() / target.size(0) * 100
    adv_acc = pred_adv.eq(target).sum().item() / target.size(0) * 100

print(f"✅ Clean Accuracy: {clean_acc:.2f}%")
print(f"✅ Adversarial Accuracy (FGSM): {adv_acc:.2f}%")
print(f"✅ Attack Success: {clean_acc - adv_acc:.2f}% drop")

✅ Clean Accuracy: 40.62%
✅ Adversarial Accuracy (FGSM): 21.88%
✅ Attack Success: 18.75% drop


In [ ]:
# @title Test Stronger PGD Attack
print("Testing stronger PGD attack...")

# Create a more powerful attack
pgd_attack = ProjectedGradientDescent(
    estimator=art_classifier,
    eps=0.3,  # Larger perturbation
    eps_step=0.01,
    max_iter=20,  # More iterations
    verbose=False
)

data_adv_pgd_np = pgd_attack.generate(x=data.cpu().numpy())
data_adv_pgd = torch.from_numpy(data_adv_pgd_np).to(device)

# Check accuracy
with torch.no_grad():
    outputs_adv_pgd = model(data_adv_pgd)
    _, pred_adv_pgd = outputs_adv_pgd.max(1)
    adv_pgd_acc = pred_adv_pgd.eq(target).sum().item() / target.size(0) * 100

print(f"✅ Clean Accuracy: {clean_acc:.2f}%")
print(f"✅ FGSM Accuracy (ε=0.1): {adv_acc:.2f}%")
print(f"✅ PGD Accuracy (ε=0.3): {adv_pgd_acc:.2f}%")
print(f"✅ PGD Attack Success: {clean_acc - adv_pgd_acc:.2f}% drop")

Testing stronger PGD attack...
✅ Clean Accuracy: 40.62%
✅ FGSM Accuracy (ε=0.1): 21.88%
✅ PGD Accuracy (ε=0.3): 12.50%
✅ PGD Attack Success: 28.12% drop
